In [20]:
import os
import json
import pandas as pd

# Extracting everything in the folder B-2224220025
folder_path = r"C:\Users\dsai3\Downloads\B-2224220025 (1)\B-2224220025"
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# Intitalize Dataframe
all_dfs = []

# Loop through each file and process it
for file_name in json_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        data = json.load(file)
        df = pd.json_normalize(data['interval_records'])
        df['source_file'] = file_name
        all_dfs.append(df)

# Combine all data into one DataFrame
combined_df = pd.concat(all_dfs, ignore_index=True)

#Convert timestamp to datetime
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])

# Display the result
print(combined_df.head())
print(len(combined_df))
combined_df.to_csv("apartment_records.csv", index=False, encoding='utf-8-sig')
#print(f"Excel File Created: {output_file}")

# Extracting everything in the folder B-2789678491
folder_path = r"C:\Users\dsai3\Downloads\B-2789678491 (1)\B-2789678491"

# Lists to store records
interval_dfs = []
register_dfs = []

# Loop through each JSON file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as f:
            data = json.load(f)

            if data.get("interval_records"):
                df_interval = pd.json_normalize(data['interval_records'])
                df_interval['source_file'] = filename
                interval_dfs.append(df_interval)
            if data.get("register_records"):
                df_register = pd.json_normalize(data['register_records'])
                df_register['source_file'] = filename
                register_dfs.append(df_register)

# Combine all data
combined_interval_df = pd.concat(interval_dfs, ignore_index=True) if interval_dfs else pd.DataFrame()
combined_register_df = pd.concat(register_dfs, ignore_index=True) if register_dfs else pd.DataFrame()

# Optional: convert timestamp columns
if not combined_interval_df.empty:
    combined_interval_df['timestamp'] = pd.to_datetime(combined_interval_df['timestamp'])

if not combined_register_df.empty:
    combined_register_df['timestamp'] = pd.to_datetime(combined_register_df['timestamp'])



print("Interval Records:", combined_interval_df.shape)
print(combined_interval_df.head())
print("Register Records:", combined_register_df.shape)
print(combined_register_df.head())

# Save to CSV if needed
combined_interval_df.to_csv("combined_interval_records.csv", index=False, encoding='utf-8-sig')
combined_register_df.to_csv("combined_register_records.csv", index=False, encoding='utf-8-sig')


                  timestamp  measurement  uom attributes.Serial  \
0 2025-05-18 00:15:00+00:00     0.013578  2.0          70028794   
1 2025-05-18 00:30:00+00:00     0.039125  2.0          70028794   
2 2025-05-18 00:45:00+00:00     0.032656  2.0          70028794   
3 2025-05-18 01:00:00+00:00     0.014164  2.0          70028794   
4 2025-05-18 01:15:00+00:00     0.014125  2.0          70028794   

  attributes.Phase attributes.Type attributes.TOU attributes.Unit  \
0               11       Intervals              0              kW   
1               11       Intervals              0              kW   
2               11       Intervals              0              kW   
3               11       Intervals              0              kW   
4               11       Intervals              0              kW   

                  source_file  
0  B-2224220025_20250518.json  
1  B-2224220025_20250518.json  
2  B-2224220025_20250518.json  
3  B-2224220025_20250518.json  
4  B-2224220025_202505

In [21]:
#Looking at the unique timestamps
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], errors='coerce')
combined_df['date'] = combined_df['timestamp'].dt.date
unique_dates = combined_df['date'].dropna().unique()
unique_dates = sorted(unique_dates)

print("Unique Dates in the Dataset:")
for date in unique_dates:
    print(date)
print(combined_df.isnull().sum())

combined_df['measurement'].describe()
combined_df[combined_df['measurement'] == 0]
combined_df[combined_df['measurement'] < 0]
combined_df[combined_df['measurement'] > combined_df['measurement'].quantile(0.999)]

from scipy import stats

z_scores = stats.zscore(combined_df['measurement'])
outliers_z = combined_df[abs(z_scores) > 3]  # Anything > 3 std dev
print(outliers_z)

Unique Dates in the Dataset:
2025-05-18
2025-05-19
2025-05-20
2025-05-21
2025-05-22
2025-05-23
2025-05-24
2025-05-25
2025-05-26
2025-05-27
2025-05-28
2025-05-29
2025-05-30
2025-05-31
2025-06-01
2025-06-02
2025-06-03
2025-06-04
2025-06-05
2025-06-06
2025-06-07
2025-06-08
2025-06-09
2025-06-10
2025-06-11
2025-06-12
2025-06-13
timestamp            0
measurement          0
uom                  0
attributes.Serial    0
attributes.Phase     0
attributes.Type      0
attributes.TOU       0
attributes.Unit      0
source_file          0
date                 0
dtype: int64
                     timestamp  measurement  uom attributes.Serial  \
208  2025-05-20 04:15:00+00:00      0.51850  2.0          70028794   
209  2025-05-20 04:30:00+00:00      0.33200  2.0          70028794   
282  2025-05-20 22:45:00+00:00      0.34225  2.0          70028794   
283  2025-05-20 23:00:00+00:00      0.34775  2.0          70028794   
284  2025-05-20 23:15:00+00:00      0.38175  2.0          70028794   
...        

In [24]:
#Looking at the unique timestamps
combined_interval_df['timestamp'] = pd.to_datetime(combined_interval_df['timestamp'], errors='coerce')
combined_interval_df['date'] = combined_interval_df['timestamp'].dt.date
unique_dates = combined_interval_df['date'].dropna().unique()
unique_dates = sorted(unique_dates)

print("Unique Dates in the Dataset:")
for date in unique_dates:
    print(date)
print(combined_interval_df.isnull().sum())

z_scores = stats.zscore(combined_register_df['measurement'])
outliers_z = combined_register_df[abs(z_scores) > 3]  # Anything > 3 std dev
print(outliers_z)

Unique Dates in the Dataset:
2025-05-18
2025-05-19
2025-05-20
2025-05-21
2025-05-22
2025-05-23
2025-05-24
2025-05-25
2025-05-26
2025-05-27
2025-05-28
2025-05-29
2025-05-30
2025-05-31
2025-06-01
2025-06-02
2025-06-03
2025-06-04
2025-06-05
2025-06-06
2025-06-07
2025-06-08
2025-06-09
2025-06-10
2025-06-11
2025-06-12
2025-06-13
timestamp            0
measurement          0
uom                  0
attributes.Serial    0
attributes.Phase     0
attributes.Type      0
attributes.TOU       0
attributes.Unit      0
source_file          0
date                 0
dtype: int64
Empty DataFrame
Columns: [timestamp, measurement, uom, attributes.Serial, attributes.Phase, attributes.Type, attributes.TOU, attributes.Unit, source_file]
Index: []


In [26]:
#Looking at the unique timestamps
combined_register_df['timestamp'] = pd.to_datetime(combined_register_df['timestamp'], errors='coerce')
combined_register_df['date'] = combined_register_df['timestamp'].dt.date
unique_dates = combined_register_df['date'].dropna().unique()
unique_dates = sorted(unique_dates)

print("Unique Dates in the Dataset:")
for date in unique_dates:
    print(date)
print(combined_register_df.isnull().sum())

z_scores = stats.zscore(combined_register_df['measurement'])
outliers_z = combined_register_df[abs(z_scores) > 3]  # Anything > 3 std dev
print(outliers_z)

Unique Dates in the Dataset:
2025-05-18
2025-05-19
2025-05-20
2025-05-21
2025-05-22
2025-05-23
2025-05-24
2025-05-25
2025-05-26
2025-05-27
2025-05-28
2025-05-29
2025-05-30
2025-05-31
2025-06-01
2025-06-02
2025-06-03
2025-06-04
2025-06-05
2025-06-06
2025-06-08
2025-06-09
2025-06-10
2025-06-11
2025-06-12
2025-06-13
2025-06-14
2025-06-15
2025-06-16
timestamp            0
measurement          0
uom                  0
attributes.Serial    0
attributes.Phase     0
attributes.Type      0
attributes.TOU       0
attributes.Unit      0
source_file          0
date                 0
dtype: int64
Empty DataFrame
Columns: [timestamp, measurement, uom, attributes.Serial, attributes.Phase, attributes.Type, attributes.TOU, attributes.Unit, source_file, date]
Index: []


In [28]:
print(combined_df.duplicated().sum())
print(combined_register_df.duplicated().sum())
print(combined_interval_df.duplicated().sum())


0
0
0


In [39]:
import pandas as pd

# Assuming df is your DataFrame
# Columns needed: 'timestamp', 'apartment_id' (e.g., Serial), etc.

# Convert timestamp to datetime
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])

# Extract date and floor timestamp to nearest 15 minutes
combined_df['date'] = combined_df['timestamp'].dt.date
combined_df['rounded_time'] = combined_df['timestamp'].dt.floor('15min')

# Create expected 15-min interval index for one full day
start_time = pd.Timestamp("00:00:00")
end_time = pd.Timestamp("23:45:00")
expected_times = pd.date_range(start=start_time, end=end_time, freq='15min').time
expected_count = len(expected_times)  # Should be 96 for 15-min intervals

# Function to check missing timestamps for one group
def find_missing_intervals(group):
    actual_times = group['rounded_time'].dt.time
    missing = set(expected_times) - set(actual_times)
    return pd.Series({
        'missing_count': len(missing),
        'missing_times': sorted(list(missing)) if missing else None
    })

# Apply group check
missing_summary = combined_df.groupby(['attributes.Serial', 'date']).apply(find_missing_intervals).reset_index()

# Filter for groups with missing timestamps
missing_summary = missing_summary[missing_summary['missing_count'] > 0]

print(missing_summary)
missing_summary.to_csv("missing_summary.csv", index=False, encoding='utf-8-sig')

   attributes.Serial        date  missing_count  \
0           70028794  2025-05-18            1.0   
20          70028794  2025-06-07           95.0   
21          70028794  2025-06-08            1.0   
26          70028794  2025-06-13           95.0   

                                        missing_times  
0                                          [00:00:00]  
20  [00:15:00, 00:30:00, 00:45:00, 01:00:00, 01:15...  
21                                         [00:00:00]  
26  [00:15:00, 00:30:00, 00:45:00, 01:00:00, 01:15...  


C:\Users\dsai3\AppData\Local\Temp\ipykernel_39908\3872775335.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_summary = combined_df.groupby(['attributes.Serial', 'date']).apply(find_missing_intervals).reset_index()
